# BUSINESS UNDERSTANDING

## Overview

Road accidents are a common occurrence in our daily lives, and it is likely that you or someone you know has been involved in one, hopefully without severe consequences. The ultimate goal is to reduce the number of road accidents. We will work with the Chicago Car Crash dataset to identify trends and predict the major causes of road accidents. By analyzing this data, we aim to develop strategies to reduce the frequency and severity of road accidents.

## Business Problem 

For this project our stakeholders are NHTSA (National Highway Traffic Safety Administration) which is the governing body in the United States of America that deals with road safety. Our objectives for our stakeholders is to explain to them:

    1. Predict the major cause of road accidents in Chicago

    2. Identify the common trends of road accidents
    
    3. Analyze the Temporal Distribution of Accidents, identify peak times and days of the week when accidents are most likely to occur.



## DATA UNDERSTANDING

For this project the dataset we will be using is the Chicago Car Crashes Dataset. This data set includes two csv files, Traffic Crashes - People and Traffic Crashes - Vehicles

In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
from datetime import datetime

In [2]:
#Loading the data
df1 = pd.read_csv("C:\\Users\\willi\\OneDrive\\Documents\\GitHub\\DSC-Phase-3-project\\Traffic_Crashes_-_People_20240603.csv")
df2 = pd.read_csv("C:\\Users\\willi\\OneDrive\\Documents\\GitHub\\DSC-Phase-3-project\\Traffic_Crashes_-_Vehicles_20240603.csv")


C:\Users\willi\AppData\Local\Temp\ipykernel_28368\553886820.py:2: DtypeWarning: Columns (19,23,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("C:\\Users\\willi\\OneDrive\\Documents\\GitHub\\DSC-Phase-3-project\\Traffic_Crashes_-_People_20240603.csv")
C:\Users\willi\AppData\Local\Temp\ipykernel_28368\553886820.py:3: DtypeWarning: Columns (20,39,40,41,43,47,48,49,52,54,57,58,59,60,67,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("C:\\Users\\willi\\OneDrive\\Documents\\GitHub\\DSC-Phase-3-project\\Traffic_Crashes_-_Vehicles_20240603.csv")


In [3]:
df1.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,M,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,M,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [4]:
df2.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1717556,7b1763088507f77e0e552c009a6bf89a4d6330c7527706...,12/06/2023 03:24:00 PM,1,DRIVER,NaN,1634931.0,NaN,NISSAN,SENTRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1717574,2603ff5a88f0b9b54576934c5ed4e4a64e8278e005687b...,12/06/2023 04:00:00 PM,2,DRIVER,NaN,1634978.0,NaN,CHRYSLER,SEBRING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1717579,a52ef70e33d468b855b5be44e8638a564434dcf99c0edf...,12/06/2023 04:30:00 PM,1,DRIVER,NaN,1634948.0,NaN,SUBARU,OUTBACK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1720118,609055f4b1a72a44d6ec40ba9036cefd7c1287a755eb6c...,12/10/2023 12:12:00 PM,1,DRIVER,NaN,1637401.0,NaN,TOYOTA,RAV4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1720119,609055f4b1a72a44d6ec40ba9036cefd7c1287a755eb6c...,12/10/2023 12:12:00 PM,2,DRIVER,NaN,1637408.0,NaN,SUBARU,OUTBACK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Understanding the dataset by getting the names of all columns and data types
print("Column names for DataFrame 1:")
print(list(df1.columns))

print("\nDataFrame 1 Information:")
df1.info()


Column names for DataFrame 1:
['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'VEHICLE_ID', 'CRASH_DATE', 'SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'EJECTION', 'INJURY_CLASSIFICATION', 'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION', 'DRIVER_VISION', 'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION', 'BAC_RESULT', 'BAC_RESULT VALUE', 'CELL_PHONE_USE']

DataFrame 1 Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1843430 entries, 0 to 1843429
Data columns (total 29 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   PERSON_ID              object 
 1   PERSON_TYPE            object 
 2   CRASH_RECORD_ID        object 
 3   VEHICLE_ID             float64
 4   CRASH_DATE             object 
 5   SEAT_NO                float64
 6   CITY                   object 
 7   STATE                  object 
 8   

## Df1 column names and meaning
PERSON_ID: A unique identifier for each person record. IDs starting with 'P' indicate passengers, while IDs starting with 'O' indicate non-passengers (e.g., drivers, pedestrians, cyclists).

PERSON_TYPE: Type of roadway user involved in the crash.

CRASH_RECORD_ID: An identifier that links to the same crash in the Crashes and Vehicles datasets. Also serves as a unique ID in the Crashes dataset.

VEHICLE_ID: Corresponding CRASH_UNIT_ID from the Vehicles dataset.

CRASH_DATE: Date and time of the crash as entered by the reporting officer.

SEAT_NO: Code indicating the seating position of motor vehicle occupants.

CITY: City of residence of the person involved in the crash.

STATE: State of residence of the person involved in the crash.

ZIPCODE: ZIP Code of residence of the person involved in the crash.

SEX: Gender of the person involved in the crash, as determined by the reporting officer.

AGE: Age of the person involved in the crash.

DRIVERS_LICENSE_STATE: State issuing the driver's license of the person involved in the crash.

DRIVERS_LICENSE_CLASS: Class of driver's license of the person involved in the crash.

SAFETY_EQUIPMENT: Safety equipment used by the vehicle occupant in the crash, if any.

AIRBAG_DEPLOYED: Whether the vehicle occupant airbag deployed as a result of the crash.

EJECTION: Whether the vehicle occupant was ejected or extricated from the vehicle as a result of the crash.

INJURY_CLASSIFICATION: Severity of injury sustained by the person in the crash.

HOSPITAL: Hospital to which the injured person was taken.

EMS_AGENCY: EMS agency that transported the injured person to the hospital.

EMS_RUN_NO: EMS agency run number.

DRIVER_ACTION: Driver action that contributed to the crash, as determined by the reporting officer.

DRIVER_VISION: Objects that obscured the driver’s vision at the time of the crash.

PHYSICAL_CONDITION: Driver’s apparent physical condition at the time of the crash, as observed by the reporting officer.

PEDPEDAL_ACTION: Action of pedestrian or cyclist at the time of the crash.

PEDPEDAL_VISIBILITY: Visibility of pedestrian or cyclist safety equipment in use at the time of the crash.

PEDPEDAL_LOCATION: Location of pedestrian or cyclist at the time of the crash.

BAC_RESULT: Status of blood alcohol concentration testing for the driver or other person involved in the crash.

BAC_RESULT_VALUE: Driver’s blood alcohol concentration test result.

CELL_PHONE_USE: Whether the person was using a cellphone at the time of the crash, as determined by the reporting officer.

In [6]:
#Understanding the dataset by getting the names of all columns and data types
print("Column names for DataFrame 2:")
print(list(df2.columns))

print("\nDataFrame 1 Information:")
df2.info()


Column names for DataFrame 2:
['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'CRASH_DATE', 'UNIT_NO', 'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE', 'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER', 'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY', 'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I', 'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I', 'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I', 'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO', 'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE', 'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO', 'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO', 'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I', 'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I', 'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH', 'TRAILER2_LENGTH', 'TOTAL_VEH

## DF 2 Column names and Meanings
CRASH_UNIT_ID: A unique identifier for each vehicle record.

CRASH_RECORD_ID: An identifier that links to the same crash in the Crashes and People datasets. Also serves as a unique ID in the Crashes dataset.

CRASH_DATE: Date and time of the crash as entered by the reporting officer.

UNIT_NO: A unique ID for each unit within a specific crash report.

UNIT_TYPE: The type of unit.

NUM_PASSENGERS: Number of passengers in the vehicle. Excluding the driver.

VEHICLE_ID: A unique identifier for each vehicle.

CMRC_VEH_I:

MAKE: The make (brand) of the vehicle, if relevant.

MODEL: The model of the vehicle, if relevant.

LIC_PLATE_STATE: The state issuing the license plate of the vehicle, if relevant.

VEHICLE_YEAR: The model year of the vehicle, if relevant.

VEHICLE_DEFECT:

VEHICLE_TYPE: The type of vehicle, if relevant.

VEHICLE_USE: The normal use of the vehicle, if relevant.

TRAVEL_DIRECTION: The direction in which the unit was traveling prior to the crash, as determined by the reporting officer.

MANEUVER: The action the unit was taking prior to the crash, as determined by the reporting officer.

TOWED_I: Indicator of whether the vehicle was towed.

FIRE_I:

OCCUPANT_CNT: The number of people in the unit, as determined by the reporting officer.

EXCEED_SPEED_LIMIT_I: Indicator of whether the unit was speeding, as determined by the reporting officer.

TOWED_BY: Entity that towed the unit, if relevant.

TOWED_TO: Location to which the unit was towed, if relevant.

AREA_00_I:

AREA_01_I:

AREA_02_I:

AREA_03_I:

AREA_04_I:

AREA_05_I:

AREA_06_I:

AREA_07_I:

AREA_08_I:

AREA_09_I:

AREA_10_I:

AREA_11_I:

AREA_12_I:

AREA_99_I:

FIRST_CONTACT_POINT:

CMV_ID:

USDOT_NO:

CCMC_NO:

ILCC_NO:

COMMERCIAL_SRC:

GVWR:

CARRIER_NAME:

CARRIER_STATE:

CARRIER_CITY:

HAZMAT_PLACARDS_I:

HAZMAT_NAME:

UN_NO:

HAZMAT_PRESENT_I:

HAZMAT_REPORT_I:

HAZMAT_REPORT_NO:

MCS_REPORT_I:

MCS_REPORT_NO:

HAZMAT_VIO_CAUSE_CRASH_I:

MCS_VIO_CAUSE_CRASH_I:

IDOT_PERMIT_NO:

WIDE_LOAD_I:

TRAILER1_WIDTH:

TRAILER2_WIDTH:

TRAILER1_LENGTH:

TRAILER2_LENGTH:

TOTAL_VEHICLE_LENGTH:

AXLE_CNT:

VEHICLE_CONFIG:

CARGO_BODY_TYPE:

LOAD_TYPE:

HAZMAT_OUT_OF_SERVICE_I:

MCS_OUT_OF_SERVICE_I:

HAZMAT_CLASS:


As you can see this particular dataset has a lof of columns with no meaning. What this means is that as we move along many of it's columns will need to be dropped

# Data Preparation

In [7]:
# Next is to check for any missing values in the datasets
df1.isnull().sum()

PERSON_ID                      0
PERSON_TYPE                    0
CRASH_RECORD_ID                0
VEHICLE_ID                 36954
CRASH_DATE                     0
SEAT_NO                  1470569
CITY                      501732
STATE                     481754
ZIPCODE                   611397
SEX                        30712
AGE                       537851
DRIVERS_LICENSE_STATE     763525
DRIVERS_LICENSE_CLASS     941650
SAFETY_EQUIPMENT            5150
AIRBAG_DEPLOYED            35553
EJECTION                   22908
INJURY_CLASSIFICATION        733
HOSPITAL                 1539920
EMS_AGENCY               1655687
EMS_RUN_NO               1812202
DRIVER_ACTION             376163
DRIVER_VISION             376719
PHYSICAL_CONDITION        375165
PEDPEDAL_ACTION          1808230
PEDPEDAL_VISIBILITY      1808298
PEDPEDAL_LOCATION        1808232
BAC_RESULT                375187
BAC_RESULT VALUE         1841388
CELL_PHONE_USE           1842271
dtype: int64

As you can see from above we have a lot of missing data that we have to work on. Some columns have a significant amount of missing values and it would be better to just drop them so for this we will include a threshold of 50%. If a column has more than 50% of it's data missing it will be dropped. We also have some missing values so we will also drop rows with missing values, like CRASH_RECORD_ID, PERSON_ID, and CRASH_DATE.

In [8]:
# Drop columns with too many missing values
missing_values = df1.isnull().sum()
threshold = 0.5
columns_to_drop = missing_values[missing_values > threshold * len(df1)].index
df1.drop(columns=columns_to_drop, inplace=True)

# Drop rows with missing target values or critical features
df1.dropna(subset=['CRASH_RECORD_ID', 'PERSON_ID', 'CRASH_DATE'], inplace=True)

# Impute missing values in numerical columns
df1[df1.select_dtypes(include=['number']).columns] = SimpleImputer(strategy='median').fit_transform(df1.select_dtypes(include=['number']))

# Impute missing values in categorical columns
df1[df1.select_dtypes(include=['object']).columns] = SimpleImputer(strategy='most_frequent').fit_transform(df1.select_dtypes(include=['object']))

df1.drop_duplicates(inplace=True)

df1.isnull().sum()


PERSON_ID                0
PERSON_TYPE              0
CRASH_RECORD_ID          0
VEHICLE_ID               0
CRASH_DATE               0
CITY                     0
STATE                    0
ZIPCODE                  0
SEX                      0
AGE                      0
DRIVERS_LICENSE_STATE    0
SAFETY_EQUIPMENT         0
AIRBAG_DEPLOYED          0
EJECTION                 0
INJURY_CLASSIFICATION    0
DRIVER_ACTION            0
DRIVER_VISION            0
PHYSICAL_CONDITION       0
BAC_RESULT               0
dtype: int64

Now we do the same for our other dataset

In [9]:
df2.isnull().sum()

CRASH_UNIT_ID                    0
CRASH_RECORD_ID                  0
CRASH_DATE                       0
UNIT_NO                          0
UNIT_TYPE                     2172
                            ...   
CARGO_BODY_TYPE            1698869
LOAD_TYPE                  1699477
HAZMAT_OUT_OF_SERVICE_I    1700650
MCS_OUT_OF_SERVICE_I       1700403
HAZMAT_CLASS               1711530
Length: 71, dtype: int64

In [10]:
# Drop columns with too many missing values
missing_values2 = df2.isnull().sum()
threshold = 0.5
columns_to_drop = missing_values2[missing_values2 > threshold * len(df2)].index
df2.drop(columns=columns_to_drop, inplace=True)


# For remaining missing values, decide on imputation strategy
num_cols = df2.select_dtypes(include=['number']).columns
cat_cols = df2.select_dtypes(include=['object']).columns

num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Apply imputers and drop duplicates
df2[num_cols] = num_imputer.fit_transform(df2[num_cols])
df2[cat_cols] = cat_imputer.fit_transform(df2[cat_cols])
df2.drop_duplicates(inplace=True)

#Convert date columns to datetime
df2['CRASH_DATE'] = pd.to_datetime(df2['CRASH_DATE'])


df2.isnull().sum()

C:\Users\willi\AppData\Local\Temp\ipykernel_28368\120631478.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['CRASH_DATE'] = pd.to_datetime(df2['CRASH_DATE'])


CRASH_UNIT_ID          0
CRASH_RECORD_ID        0
CRASH_DATE             0
UNIT_NO                0
UNIT_TYPE              0
VEHICLE_ID             0
MAKE                   0
MODEL                  0
LIC_PLATE_STATE        0
VEHICLE_YEAR           0
VEHICLE_DEFECT         0
VEHICLE_TYPE           0
VEHICLE_USE            0
TRAVEL_DIRECTION       0
MANEUVER               0
OCCUPANT_CNT           0
FIRST_CONTACT_POINT    0
dtype: int64

Next in order to save on time, and to make it easier to go through this file we will merge the two datasets in order to have all the information we need in one place. As we can see that both crash_record_id and vehicle_id exist in both datasets it is the common key that we will use to merge the datasets

In [19]:
merged_df = pd.merge(df1, df2, on=['CRASH_RECORD_ID', 'VEHICLE_ID'], how='inner')
merged_df.columns

Index(['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'VEHICLE_ID',
       'CRASH_DATE_x', 'CITY', 'STATE', 'ZIPCODE', 'SEX', 'AGE',
       'DRIVERS_LICENSE_STATE', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED',
       'EJECTION', 'INJURY_CLASSIFICATION', 'DRIVER_ACTION', 'DRIVER_VISION',
       'PHYSICAL_CONDITION', 'BAC_RESULT', 'CRASH_UNIT_ID', 'CRASH_DATE_y',
       'UNIT_NO', 'UNIT_TYPE', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
       'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRECTION', 'MANEUVER', 'OCCUPANT_CNT', 'FIRST_CONTACT_POINT'],
      dtype='object')

Now we need to drop unnecessary columns from this dataset, as it is the final dataset that will be used for modelling. So we will only keep the necessary columns that may help make our model more accurate. Columns like CRASH_RECORD_ID or VEHICLE_ID are not necessary when modelling. The city and state columns are also not needed because all the accidents captured in this data happened in the same region. We also have some duplicate columns that need to be dropped.

In [20]:
#Drop unnecessary columns
up_merged_df = merged_df.drop(columns=['CRASH_RECORD_ID', 'VEHICLE_ID', 'CITY', 'STATE', 'FIRST_CONTACT_POINT', 'CRASH_DATE_x', 'PERSON_ID', 'ZIPCODE', 'LIC_PLATE_STATE'])
up_merged_df.columns

Index(['PERSON_TYPE', 'SEX', 'AGE', 'DRIVERS_LICENSE_STATE',
       'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED', 'EJECTION',
       'INJURY_CLASSIFICATION', 'DRIVER_ACTION', 'DRIVER_VISION',
       'PHYSICAL_CONDITION', 'BAC_RESULT', 'CRASH_UNIT_ID', 'CRASH_DATE_y',
       'UNIT_NO', 'UNIT_TYPE', 'MAKE', 'MODEL', 'VEHICLE_YEAR',
       'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION',
       'MANEUVER', 'OCCUPANT_CNT'],
      dtype='object')